In [ ]:
import micropip
import pyodide_js
await pyodide_js.loadPackage("https://makslevental.github.io/mlir-python-wasm/files/mlir_python_bindings-21.0.0.2025062201-cp312-cp312-pyodide_2024_0_wasm32.whl")
await micropip.install("emfs:mlir_python_extras-0.0.8.2025062302-py3-none-any.whl")
await pyodide_js.loadPackage(["numpy", "pyyaml"])

In [ ]:
import numpy as np

import mlir.extras.types as T
from mlir.extras.ast.canonicalize import canonicalize
from mlir.extras.context import mlir_mod_ctx
from mlir.extras.dialects.ext.arith import constant
from mlir.extras.dialects.ext.memref import S
from mlir.extras.dialects.ext.func import func
from mlir.extras.dialects.ext.scf import canonicalizer as scf, range_
from mlir.extras.runtime.passes import Pipeline, run_pipeline
from mlir.extras.runtime.refbackend import LLVMJITBackend
from mlir.ir import StridedLayoutAttr

# you need this to register the memref value caster
# noinspection PyUnresolvedReferences
import mlir.extras.dialects.ext.memref

ctx_man = mlir_mod_ctx()
ctx = ctx_man.__enter__()

In [ ]:
K = 256
D = 32

F = K // D
ranked_memref_kxk_f32 = T.memref(K, K, T.f32())
layout = StridedLayoutAttr.get(S, (K, 1))
ranked_memref_dxd_f32 = T.memref(D, D, T.f32(), layout=layout)

@func(emit=True)
@canonicalize(using=scf)
def tile(
    A: ranked_memref_dxd_f32, B: ranked_memref_dxd_f32, C: ranked_memref_dxd_f32
):
    for i in range_(0, D):
        for j in range_(0, D):
            C[i, j] = A[i, j] + B[i, j]

@func(emit=True)
@canonicalize(using=scf)
def tiled_memfoo(
    A: ranked_memref_kxk_f32, B: ranked_memref_kxk_f32, C: ranked_memref_kxk_f32
):
    for i in range_(0, F):
        for j in range_(0, F):
            l = lambda l: l * D
            r = lambda r: (r + 1) * D
            a, b, c = (
                A[l(i) : r(i), l(j) : r(j)],
                B[l(i) : r(i), l(j) : r(j)],
                C[l(i) : r(i), l(j) : r(j)],
            )
            tile(a, b, c)

In [ ]:
module = run_pipeline(ctx.module, str(Pipeline().cse()))
print(module)
ctx_man.__exit__(None, None, None);